In [1]:
from __future__ import print_function
import lasagne

Using gpu device 0: GeForce GTX TITAN Black


Custom layer min reqs:
- subclass `Layer`
- implement `get_output_for()`

In [2]:
class DoubleLayer(lasagne.layers.Layer):
    def get_output_for(self, input, **kwargs):
        return 2 * input

In [3]:
# If layer changes shape of the data, implement get_output_shape_for() -> tuple of int
# e.g. layer that computes sum across trailing axis of its input
class SumLayer(lasagne.layers.Layer):
    def get_output_for(self, input, **kwargs):
        return input.sum(axis=1)
    
    def get_output_shape_for(self, input_shape):
        return input_shape[:-1]

In [4]:
# Layer parameters, should be init in the constructor
class DotLayer(lasagne.layers.Layer):
    def __init__(self, incoming, num_units, W=lasagne.init.Normal(0.01), **kwargs):
        super(DotLayer, self).__init__(incoming, **kwargs)
        num_inputs = self.input_shape[1]
        self.num_units = num_units
        self.W = self.add_param(W, (num_inputs, num_units), name='W')  # Create Theano shared var
                                                                       # retrieved with Layer.get_params()
                                                                       # or in this case self.W
    
    def get_output_for(self, input, **kwargs):  # kwargs use case: turn on/off dropout at training/testing
        return T.dot(input, self.W)
    
    def get_output_shape_for(self, input_shape):
        return (input_shape[0], self.num_units)

In [6]:
# Rough implementation of dropout
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
_srng = RandomStreams()

class DropoutLayer(lasagne.layers.Layer):
    def __init__(self, incoming, p=0.5, **kwargs):
        super(DropoutLayer, self).__init__(incoming, **kwargs)
        self.p = p
    
    def get_output_for(self, input, deterministic=False, **kwargs):
        if deterministic:
            return input
        else:
            retain_prob = 1 - self.p
            return input * _srng.binomial(input.shape, p=retain_prob, dtype=theano.config.floatX)